In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re
from konlpy.tag import Twitter
import tensorflow as tf
import math

In [2]:
# 태그 단어
PAD = "<PADDING>"   # 패딩(입력 데이터의 개수)
                    # 한문장에 단어의 개수가 30개 미만일
                    # 경우 <PADDING> 으로 채워 넣음

STA = "<START>"     # 디코더가 Answer 시작 하는 기호

END = "<END>"       # 디코더가 Answer를 끝내는 기호

OOV = "<OOV>"       # 학습한 단어가 아닌 
                    # 단어가 입력으로 들어 왔을때
                    # (Out of Vocabulary) 로 변환

In [3]:
# 태그 인덱스
# 30글자 미만의 입력이 입력 되었을때 0
# 으로 변환
PAD_INDEX = 0 

# 디코더가 Answer를 시작하는 숫자 1
STA_INDEX = 1 

# 디코더가 Answer를 끝내는 숫자 2
END_INDEX = 2 

# 입력시 학습하지 않은 단어가 입력 되었을때
# 3으로 변환
OOV_INDEX = 3 

In [4]:
# 데이터 타입
ENCODER_INPUT  = 0  # Question 입력을 처리하는 인코더의 입력
DECODER_INPUT  = 1  # Answer 입력을 처리하는 디코더의 입력
DECODER_TARGET = 2  # 실제 Answer 의 값

In [5]:
# 한 문장에서 단어 시퀀스의 최대 개수
# 한문장당 30개씩 단어를 입력 받음
max_sequences = 30

# 임베딩 벡터 차원
#embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

In [6]:
# 챗봇 데이터 로드
chatbot_data = pd.read_csv(
                            "ChatbotData.csv" , 
                             encoding='utf-8'
                          )

In [7]:
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [8]:
# chatbot_data['Q'] : 입력데이터 질문이 저장 되 있음 (type : DataFrame Seires)
# chatbot_data['Q'].tolist() : 질문을 리스트로 변환해서 question에 저장 (type : list)
question = chatbot_data['Q'].tolist()

In [9]:
question

['12시 땡!',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨',
 'SNS보면 나만 빼고 다 행복해보여',
 '가끔 궁금해',
 '가끔 뭐하는지 궁금해',
 '가끔은 혼자인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀난다',
 '가상화폐 쫄딱 망함',
 '가스불 켜고 나갔어',
 '가스불 켜놓고 나온거 같아',
 '가스비 너무 많이 나왔다.',
 '가스비 비싼데 감기 걸리겠어',
 '가스비 장난 아님',
 '가장 확실한 건 뭘까?',
 '가족 여행 가기로 했어',
 '가족 여행 고고',
 '가족 여행 어디로 가지?',
 '가족 있어?',
 '가족관계 알려 줘',
 '가족끼리 여행간다.',
 '가족들 보고 싶어',
 '가족들이랑 서먹해',
 '가족들이랑 서먹해졌어',
 '가족들이랑 어디 가지?',
 '가족들이랑 여행 갈거야',
 '가족여행 가야지',
 '가족이 누구야?',
 '가족이랑 여행 가려고',
 '가족한테 스트레스 풀었어',
 '가출할까?',
 '가출해도 갈 데가 없어',
 '간만에 떨리니까 좋더라',
 '간만에 쇼핑 중',
 '간만에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까?',
 '감 말랭이 먹고 싶다.',
 '감 말랭이 먹어야지',
 '감기 같애',
 '감기 걸린 것 같아',
 '감기 기운이 있어',
 '감기 들 거 같애',
 '감기가 오려나',
 '감기약이 없어',
 '감기인거 같애',
 '감미로운 목소리 좋아',
 '감정이 쓰레기통처럼 엉망진창이야',
 '감정컨트롤을 못하겠어',
 '감정컨트롤이 안돼',
 '감히 나를 무시하는 애가 있어',
 '갑자기 나쁜 생각이 막 들더라',
 '갑자기 눈물 나',

In [10]:
# chatbot_data['A'] : label 데이터 답변이 저장 되 있음
# chatbot_data['A'].tolist() : 답변을 리스트로 변환해서
#                             answer 에 저장
answer = chatbot_data['A'].tolist()

In [11]:
answer

['하루가 또 가네요.',
 '위로해 드립니다.',
 '여행은 언제나 좋죠.',
 '여행은 언제나 좋죠.',
 '눈살이 찌푸려지죠.',
 '다시 새로 사는 게 마음 편해요.',
 '다시 새로 사는 게 마음 편해요.',
 '잘 모르고 있을 수도 있어요.',
 '시간을 정하고 해보세요.',
 '시간을 정하고 해보세요.',
 '자랑하는 자리니까요.',
 '그 사람도 그럴 거예요.',
 '그 사람도 그럴 거예요.',
 '혼자를 즐기세요.',
 '돈은 다시 들어올 거예요.',
 '땀을 식혀주세요.',
 '어서 잊고 새출발 하세요.',
 '빨리 집에 돌아가서 끄고 나오세요.',
 '빨리 집에 돌아가서 끄고 나오세요.',
 '다음 달에는 더 절약해봐요.',
 '따뜻하게 사세요!',
 '다음 달에는 더 절약해봐요.',
 '가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '더 가까워질 기회가 되겠네요.',
 '저도요.',
 '다들 바빠서 이야기할 시간이 부족했나봐요.',
 '다들 바빠서 이야기할 시간이 부족했나봐요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '좋은 생각이에요.',
 '더 가까워질 기회가 되겠네요.',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '좋은 생각이에요.',
 '정말 후회할 습관이에요.',
 '무모한 결정을 내리지 마세요.',
 '선생님이나 기관에 연락해보세요.',
 '떨리는 감정은 그 자체로 소중해요.',
 '득템했길 바라요.',
 '휴식도 필요하죠.',
 '단짠으로 두 개 사는게 진리죠.',
 '단짠으로 두

In [12]:
# 데이터 개수
len(question)

11823

In [13]:
# 챗봇 데이터 출력
for i in range(10):
    #질문 (입력 데이터)
    print('Q : ' + question[i])
    #대답 (label 데이터)
    print('A : ' + answer[i])
    print()


Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



<br>
<br>

# 단어 사전 생성

In [14]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Twitter()
    
    # 문장을 단어 단위로 잘라서 저장할 리스트
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        #[.,!?\"':;~()] : 특수 기호를 찾을 객체
        RE_FILTER = re.compile("[.,!?\"':;~()]")
        #re.sub(RE_FILTER, "", sentence) :
        # sentence에서 
        # RE_FILTER 에 설정된 특수기호를 "" 로 변환(이 경우 삭제)해서 리턴
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # tagger.morphs(sentence) : sentence를 단어 단위로
        #                          분리하여 리스트로 반환  

        # " ".join(tagger.morphs(sentence)) :    ## 리스트의 요소들 사이에 공백을 두고 합쳐서 문자열로 반환
        #                          단어와 단어 사이에 " "
        #                          추가해서 리턴                
        sentence = " ".join(tagger.morphs(sentence))

        #단어 단위로 분리된 sentence를 sentences_pos에 추가    # 단어 단위로 띄어쓰기된 문장들로 구성된 리스트 sentences_pos
        sentences_pos.append(sentence)
        
    return sentences_pos

In [15]:
# question에 특수 문자를 삭제하고
# 단어와 단어 사이에 공백 추가해서 리턴
question = pos_tag(question)

question

C:\Users\User\anaconda3\envs\chat_project_env01\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [16]:
# answer 에 특수 문자를 삭제하고
# 단어와 단어 사이에 공백 추가해서 리턴
answer = pos_tag(answer)

answer

['하루 가 또 가네요',
 '위로 해 드립니다',
 '여행 은 언제나 좋죠',
 '여행 은 언제나 좋죠',
 '눈살 이 찌푸려지죠',
 '다시 새로 사는 게 마음 편해요',
 '다시 새로 사는 게 마음 편해요',
 '잘 모르고 있을 수도 있어요',
 '시간 을 정 하고 해보세요',
 '시간 을 정 하고 해보세요',
 '자랑 하는 자리 니까 요',
 '그 사람 도 그럴 거 예요',
 '그 사람 도 그럴 거 예요',
 '혼자 를 즐기세요',
 '돈 은 다시 들어올 거 예요',
 '땀 을 식혀주세요',
 '어서 잊고 새 출발 하세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '다음 달 에는 더 절약 해봐요',
 '따뜻하게 사세요',
 '다음 달 에는 더 절약 해봐요',
 '가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '더 가까워질 기회 가 되겠네요',
 '저 도 요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '좋은 생각 이에요',
 '더 가까워질 기회 가 되겠네요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '좋은 생각 이에요',
 '정말 후회 할 습관 이에요',
 '무모한 결정 을 내 리지 마세요',
 '선생님 이나 기관 에 연락 해보세요',
 '떨리는 감정 은 그 자체 로 소중해요',
 '득템 했길 바라요

In [17]:
# 특수 문자를 삭제하고
# (단어와 단어 사이에 공백 추가 한 문자열들로 구성된 리스트) 데이터 조회
for i in range(10):
    #챗봇 질문
    print('질문 : ' + question[i])
    #챗봇 대답
    print('대답 : ' + answer[i])
    print("="*100)

질문 : 12시 땡
대답 : 하루 가 또 가네요
질문 : 1 지망 학교 떨어졌어
대답 : 위로 해 드립니다
질문 : 3 박 4일 놀러 가고 싶다
대답 : 여행 은 언제나 좋죠
질문 : 3 박 4일 정도 놀러 가고 싶다
대답 : 여행 은 언제나 좋죠
질문 : PPL 심하네
대답 : 눈살 이 찌푸려지죠
질문 : SD 카드 망가졌어
대답 : 다시 새로 사는 게 마음 편해요
질문 : SD 카드 안 돼
대답 : 다시 새로 사는 게 마음 편해요
질문 : SNS 맞팔 왜 안 하지 ㅠㅠ
대답 : 잘 모르고 있을 수도 있어요
질문 : SNS 시간 낭비 인 거 아는데 매일 하는 중
대답 : 시간 을 정 하고 해보세요
질문 : SNS 시간 낭비 인데 자꾸 보게 됨
대답 : 시간 을 정 하고 해보세요


In [18]:
# 질문 question 과 대답 answer를 저장할 리스트
sentences = []
#sentences에 question 추가
sentences.extend(question)    # question 자체가 리스트이기 때문에 []를 씌워서 넣을 필요 없음
##sentences에 answer 추가
sentences.extend(answer)

In [19]:
sentences

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [20]:
# question 과 answer 가 저장된
# sentences 에 단어들을 저장할 리스트
words = []

# question 과 answer 가 저장된
# sentences 에서 1줄 sentence에 대입
for sentence in sentences:
    # sentence.split() : sentence를 단어 별로 분리해서 리스트화
    # (리스트의 요소인 각) 단어 1개를 word에 대입
    for word in sentence.split():
        #단어를 words에 추가
        words.append(word)    # question, answer에 있는 모든 단어들을 요소로 갖는 리스트 words

In [21]:
words

['12시',
 '땡',
 '1',
 '지망',
 '학교',
 '떨어졌어',
 '3',
 '박',
 '4일',
 '놀러',
 '가고',
 '싶다',
 '3',
 '박',
 '4일',
 '정도',
 '놀러',
 '가고',
 '싶다',
 'PPL',
 '심하네',
 'SD',
 '카드',
 '망가졌어',
 'SD',
 '카드',
 '안',
 '돼',
 'SNS',
 '맞팔',
 '왜',
 '안',
 '하지',
 'ㅠㅠ',
 'SNS',
 '시간',
 '낭비',
 '인',
 '거',
 '아는데',
 '매일',
 '하는',
 '중',
 'SNS',
 '시간',
 '낭비',
 '인데',
 '자꾸',
 '보게',
 '됨',
 'SNS',
 '보면',
 '나',
 '만',
 '빼고',
 '다',
 '행복',
 '해보여',
 '가끔',
 '궁금해',
 '가끔',
 '뭐',
 '하는지',
 '궁금해',
 '가끔',
 '은',
 '혼자',
 '인게',
 '좋다',
 '가난한',
 '자의',
 '설움',
 '가만',
 '있어도',
 '땀',
 '난다',
 '가상',
 '화폐',
 '쫄딱',
 '망함',
 '가스',
 '불',
 '켜고',
 '나갔어',
 '가스',
 '불',
 '켜놓고',
 '나온거',
 '같아',
 '가스',
 '비',
 '너무',
 '많이',
 '나왔다',
 '가스',
 '비',
 '비싼데',
 '감기',
 '걸리겠어',
 '가스',
 '비',
 '장난',
 '아님',
 '가장',
 '확실한',
 '건',
 '뭘',
 '까',
 '가족',
 '여행',
 '가기',
 '로',
 '했어',
 '가족',
 '여행',
 '고고',
 '가족',
 '여행',
 '어디',
 '로',
 '가지',
 '가족',
 '있어',
 '가족',
 '관계',
 '알려',
 '줘',
 '가족',
 '끼리',
 '여행',
 '간다',
 '가족',
 '들',
 '보고',
 '싶어',
 '가족',
 '들',
 '이랑',
 '서먹해',
 '가족',
 '들',
 '이랑',
 '서먹해졌어',
 '가

In [22]:
# for word in words : words에서 단어 1개 word에 대입
# if len(word) > 0 : 글자수가 1 이상인 단어만 words에 저장
words = [word for word in words if len(word) > 0]

In [23]:
words

['12시',
 '땡',
 '1',
 '지망',
 '학교',
 '떨어졌어',
 '3',
 '박',
 '4일',
 '놀러',
 '가고',
 '싶다',
 '3',
 '박',
 '4일',
 '정도',
 '놀러',
 '가고',
 '싶다',
 'PPL',
 '심하네',
 'SD',
 '카드',
 '망가졌어',
 'SD',
 '카드',
 '안',
 '돼',
 'SNS',
 '맞팔',
 '왜',
 '안',
 '하지',
 'ㅠㅠ',
 'SNS',
 '시간',
 '낭비',
 '인',
 '거',
 '아는데',
 '매일',
 '하는',
 '중',
 'SNS',
 '시간',
 '낭비',
 '인데',
 '자꾸',
 '보게',
 '됨',
 'SNS',
 '보면',
 '나',
 '만',
 '빼고',
 '다',
 '행복',
 '해보여',
 '가끔',
 '궁금해',
 '가끔',
 '뭐',
 '하는지',
 '궁금해',
 '가끔',
 '은',
 '혼자',
 '인게',
 '좋다',
 '가난한',
 '자의',
 '설움',
 '가만',
 '있어도',
 '땀',
 '난다',
 '가상',
 '화폐',
 '쫄딱',
 '망함',
 '가스',
 '불',
 '켜고',
 '나갔어',
 '가스',
 '불',
 '켜놓고',
 '나온거',
 '같아',
 '가스',
 '비',
 '너무',
 '많이',
 '나왔다',
 '가스',
 '비',
 '비싼데',
 '감기',
 '걸리겠어',
 '가스',
 '비',
 '장난',
 '아님',
 '가장',
 '확실한',
 '건',
 '뭘',
 '까',
 '가족',
 '여행',
 '가기',
 '로',
 '했어',
 '가족',
 '여행',
 '고고',
 '가족',
 '여행',
 '어디',
 '로',
 '가지',
 '가족',
 '있어',
 '가족',
 '관계',
 '알려',
 '줘',
 '가족',
 '끼리',
 '여행',
 '간다',
 '가족',
 '들',
 '보고',
 '싶어',
 '가족',
 '들',
 '이랑',
 '서먹해',
 '가족',
 '들',
 '이랑',
 '서먹해졌어',
 '가

In [24]:
# set(words) : 리스트에서 중복된 단어 삭제
# list( set(words) ) : 중복된 단어 삭제 후 리스트로 변환
words = list(set(words))

In [25]:
words

['힘들겠네요',
 '헤어지고서',
 '옷장',
 '이었다던',
 '컸을거라',
 '미워해도',
 '평상시',
 '어때',
 '옆',
 '벗어나',
 '모르겠어요',
 '되겠네요',
 '흘리세요',
 '아껴주세요',
 '오시',
 '서운했어',
 '뒤통수',
 '갔네',
 '익숙해져',
 '나간대',
 '잡았던',
 '멀어진',
 '생각나긴나',
 '모두',
 '원동력',
 '적을하고',
 '지친듯',
 '보안',
 '초라하게',
 '때려주세요',
 '꿨는데',
 '있으면',
 '찾아오길',
 '문득',
 '애니',
 '쓰세요',
 '멈출',
 '은연',
 '힘들기만',
 '설렜을텐데',
 '하셔도',
 '독학',
 '하고있어요',
 '치고',
 '하는',
 '힘겨',
 '살릴듯',
 '키우는',
 '물어보지',
 '북극',
 '가고',
 '만하',
 '내든',
 '떨어',
 '존댓말',
 '아이돌',
 '조급해하지',
 '얽매여',
 '무너지지',
 '짜증',
 '얻는다는',
 '좋겠습니다',
 '해온',
 '팔길래',
 '엉망',
 '잊혀지질',
 '재밌겠지',
 '가져간',
 '대단하네요',
 '단거리',
 '제자리걸음',
 '믿었어',
 '버릴',
 '이었어요',
 '추우니',
 '알면서도',
 '경향',
 '일어나자마자',
 '볼',
 '했던걸',
 '중요한',
 '흐려지곤',
 '보고싶어서',
 '숨기',
 '몸짓',
 '떨려서',
 '사귄대',
 '꺽',
 '뽑은',
 '몫',
 '질질',
 '적응하고',
 '마시고',
 '멍청한',
 '두었을까',
 '허리띠',
 '지긋지긋한',
 '찾아올거라',
 '했다면',
 '자기개발',
 '좋을지도',
 '강남',
 '회원정보',
 '본지가',
 '궁금하지',
 '샀는데',
 '쉬어',
 '수학',
 '꿈이었으면',
 '놓는',
 '비판',
 '아름다워',
 '찾아가고',
 '없으면서',
 '힘드네',
 '다정했던',
 '있든',
 '아파도',
 '받아들이',
 '야한',
 '오랫동안',
 '후련하게',


In [26]:
len(words)

12652

In [27]:
# 제일 앞에 태그 단어 삽입

# PAD : 30 글자 미만의 입력일 경우 PAD => 0 으로 채워줌 (단어가 들어있지 않은 곳에)
# STA : Decode 입력 시작
# END : Decode 예측 끝
# OOV : 입력시 학습되지 않은 
#      단어가 입력되었을때 OOV 로 변환

words[:0] = [PAD, STA, END, OOV]

In [28]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '힘들겠네요',
 '헤어지고서',
 '옷장',
 '이었다던',
 '컸을거라',
 '미워해도',
 '평상시',
 '어때',
 '옆',
 '벗어나',
 '모르겠어요',
 '되겠네요',
 '흘리세요',
 '아껴주세요',
 '오시',
 '서운했어',
 '뒤통수',
 '갔네',
 '익숙해져',
 '나간대',
 '잡았던',
 '멀어진',
 '생각나긴나',
 '모두',
 '원동력',
 '적을하고',
 '지친듯',
 '보안',
 '초라하게',
 '때려주세요',
 '꿨는데',
 '있으면',
 '찾아오길',
 '문득',
 '애니',
 '쓰세요',
 '멈출',
 '은연',
 '힘들기만',
 '설렜을텐데',
 '하셔도',
 '독학',
 '하고있어요',
 '치고',
 '하는',
 '힘겨',
 '살릴듯',
 '키우는',
 '물어보지',
 '북극',
 '가고',
 '만하',
 '내든',
 '떨어',
 '존댓말',
 '아이돌',
 '조급해하지',
 '얽매여',
 '무너지지',
 '짜증',
 '얻는다는',
 '좋겠습니다',
 '해온',
 '팔길래',
 '엉망',
 '잊혀지질',
 '재밌겠지',
 '가져간',
 '대단하네요',
 '단거리',
 '제자리걸음',
 '믿었어',
 '버릴',
 '이었어요',
 '추우니',
 '알면서도',
 '경향',
 '일어나자마자',
 '볼',
 '했던걸',
 '중요한',
 '흐려지곤',
 '보고싶어서',
 '숨기',
 '몸짓',
 '떨려서',
 '사귄대',
 '꺽',
 '뽑은',
 '몫',
 '질질',
 '적응하고',
 '마시고',
 '멍청한',
 '두었을까',
 '허리띠',
 '지긋지긋한',
 '찾아올거라',
 '했다면',
 '자기개발',
 '좋을지도',
 '강남',
 '회원정보',
 '본지가',
 '궁금하지',
 '샀는데',
 '쉬어',
 '수학',
 '꿈이었으면',
 '놓는',
 '비판',
 '아름다워',
 '찾아가고',
 '없으면서',
 '힘드네',
 '다정했던',
 '있

In [29]:
# 단어 개수
len(words)

12656

In [30]:
# 단어 출력
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '힘들겠네요',
 '헤어지고서',
 '옷장',
 '이었다던',
 '컸을거라',
 '미워해도',
 '평상시',
 '어때',
 '옆',
 '벗어나',
 '모르겠어요',
 '되겠네요',
 '흘리세요',
 '아껴주세요',
 '오시',
 '서운했어',
 '뒤통수',
 '갔네',
 '익숙해져',
 '나간대',
 '잡았던',
 '멀어진',
 '생각나긴나',
 '모두',
 '원동력',
 '적을하고',
 '지친듯',
 '보안',
 '초라하게',
 '때려주세요',
 '꿨는데',
 '있으면',
 '찾아오길',
 '문득',
 '애니',
 '쓰세요',
 '멈출',
 '은연',
 '힘들기만',
 '설렜을텐데',
 '하셔도',
 '독학',
 '하고있어요',
 '치고',
 '하는',
 '힘겨',
 '살릴듯',
 '키우는',
 '물어보지',
 '북극',
 '가고',
 '만하',
 '내든',
 '떨어',
 '존댓말',
 '아이돌',
 '조급해하지',
 '얽매여',
 '무너지지',
 '짜증',
 '얻는다는',
 '좋겠습니다',
 '해온',
 '팔길래',
 '엉망',
 '잊혀지질',
 '재밌겠지',
 '가져간',
 '대단하네요',
 '단거리',
 '제자리걸음',
 '믿었어',
 '버릴',
 '이었어요',
 '추우니',
 '알면서도',
 '경향',
 '일어나자마자',
 '볼',
 '했던걸',
 '중요한',
 '흐려지곤',
 '보고싶어서',
 '숨기',
 '몸짓',
 '떨려서',
 '사귄대',
 '꺽',
 '뽑은',
 '몫',
 '질질',
 '적응하고',
 '마시고',
 '멍청한',
 '두었을까',
 '허리띠',
 '지긋지긋한',
 '찾아올거라',
 '했다면',
 '자기개발',
 '좋을지도',
 '강남',
 '회원정보',
 '본지가',
 '궁금하지',
 '샀는데',
 '쉬어',
 '수학',
 '꿈이었으면',
 '놓는',
 '비판',
 '아름다워',
 '찾아가고',
 '없으면서',
 '힘드네',
 '다정했던',
 '있

In [31]:
# 단어와 인덱스의 딕셔너리 생성
#for index, word in enumerate(words) :  words에 인덱스 추가
#                                       해서 인덱스는 index
#                                       단어는 word에 대입

# 단어 : 인덱스 리턴

word_to_index = {
    word: index for index, word in enumerate(words)
    }


In [32]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '힘들겠네요': 4,
 '헤어지고서': 5,
 '옷장': 6,
 '이었다던': 7,
 '컸을거라': 8,
 '미워해도': 9,
 '평상시': 10,
 '어때': 11,
 '옆': 12,
 '벗어나': 13,
 '모르겠어요': 14,
 '되겠네요': 15,
 '흘리세요': 16,
 '아껴주세요': 17,
 '오시': 18,
 '서운했어': 19,
 '뒤통수': 20,
 '갔네': 21,
 '익숙해져': 22,
 '나간대': 23,
 '잡았던': 24,
 '멀어진': 25,
 '생각나긴나': 26,
 '모두': 27,
 '원동력': 28,
 '적을하고': 29,
 '지친듯': 30,
 '보안': 31,
 '초라하게': 32,
 '때려주세요': 33,
 '꿨는데': 34,
 '있으면': 35,
 '찾아오길': 36,
 '문득': 37,
 '애니': 38,
 '쓰세요': 39,
 '멈출': 40,
 '은연': 41,
 '힘들기만': 42,
 '설렜을텐데': 43,
 '하셔도': 44,
 '독학': 45,
 '하고있어요': 46,
 '치고': 47,
 '하는': 48,
 '힘겨': 49,
 '살릴듯': 50,
 '키우는': 51,
 '물어보지': 52,
 '북극': 53,
 '가고': 54,
 '만하': 55,
 '내든': 56,
 '떨어': 57,
 '존댓말': 58,
 '아이돌': 59,
 '조급해하지': 60,
 '얽매여': 61,
 '무너지지': 62,
 '짜증': 63,
 '얻는다는': 64,
 '좋겠습니다': 65,
 '해온': 66,
 '팔길래': 67,
 '엉망': 68,
 '잊혀지질': 69,
 '재밌겠지': 70,
 '가져간': 71,
 '대단하네요': 72,
 '단거리': 73,
 '제자리걸음': 74,
 '믿었어': 75,
 '버릴': 76,
 '이었어요': 77,
 '추우니': 78,
 '알면서도': 79,
 '경향': 80,
 '일어나자마자

In [33]:
# 단어와 인덱스의 딕셔너리 생성
#for index, word in enumerate(words) :words에 인덱스 추가
#                                  해서 인덱스는 index
#                                  단어는 word에 대입


# 인덱스: 단어 리턴 [key: index, value: word]
index_to_word = {index: word for index, word in enumerate(words)}

In [34]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '힘들겠네요',
 5: '헤어지고서',
 6: '옷장',
 7: '이었다던',
 8: '컸을거라',
 9: '미워해도',
 10: '평상시',
 11: '어때',
 12: '옆',
 13: '벗어나',
 14: '모르겠어요',
 15: '되겠네요',
 16: '흘리세요',
 17: '아껴주세요',
 18: '오시',
 19: '서운했어',
 20: '뒤통수',
 21: '갔네',
 22: '익숙해져',
 23: '나간대',
 24: '잡았던',
 25: '멀어진',
 26: '생각나긴나',
 27: '모두',
 28: '원동력',
 29: '적을하고',
 30: '지친듯',
 31: '보안',
 32: '초라하게',
 33: '때려주세요',
 34: '꿨는데',
 35: '있으면',
 36: '찾아오길',
 37: '문득',
 38: '애니',
 39: '쓰세요',
 40: '멈출',
 41: '은연',
 42: '힘들기만',
 43: '설렜을텐데',
 44: '하셔도',
 45: '독학',
 46: '하고있어요',
 47: '치고',
 48: '하는',
 49: '힘겨',
 50: '살릴듯',
 51: '키우는',
 52: '물어보지',
 53: '북극',
 54: '가고',
 55: '만하',
 56: '내든',
 57: '떨어',
 58: '존댓말',
 59: '아이돌',
 60: '조급해하지',
 61: '얽매여',
 62: '무너지지',
 63: '짜증',
 64: '얻는다는',
 65: '좋겠습니다',
 66: '해온',
 67: '팔길래',
 68: '엉망',
 69: '잊혀지질',
 70: '재밌겠지',
 71: '가져간',
 72: '대단하네요',
 73: '단거리',
 74: '제자리걸음',
 75: '믿었어',
 76: '버릴',
 77: '이었어요',
 78: '추우니',
 79: '알면서도',
 80: '경향',
 81: '일어

<br>
<br>

# 전처리

In [35]:
# 문장의 단어를 숫자로 변환
def convert_text_to_index(sentences, vocabulary, type):     # [주의] question과 answer가 합쳐진 sentences가 아님
    #모든 줄의 단어를 숫자로 변환한 데이터를 저장할 리스트
    sentences_index = []
    
    # sentences 에서 1줄 sentence에 대입
    for sentence in sentences:
        # 1줄 의 단어를 숫자로 변환해서 저장 할 리스트
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])    # extend의 매개변수는 리스트
        
        # for word in sentence.split() : 단어 1개씩  리턴해서 변수 word에 저장
        for word in sentence.split():
            #사전에 있는 단어면
            if vocabulary.get(word) is not None:
                #  해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # DECODER_TARGET : label 데이터 일 경우 최대 길이 검사
        if type == DECODER_TARGET:
            # len(sentence_index) : 문장을 숫자로 변환한 데이터의 길이(리스트의 원소 개수)
            # max_sequences : 최대 입력 길이 (30)    # 한 문장의 단어들을 인덱스로 변환한 원소들로 구성된 리스트의 최대 길이는 30
            if len(sentence_index) >= max_sequences:    # DECODER_TARGET은 마지막에 <END>를 포함시켜야 하므로 실질적인 최대 길이는 29
                #sentence_index[:max_sequences-1] : 인덱스 0이상 ~ 29미만까지 자르고
                # [vocabulary[END] : 마지막에 END 추가
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]    # 리스트에 요소를 +로 추가하고 싶다면 [] 씌워주기
            else:
                # sentence_index 마지막에 END 추가
                sentence_index += [vocabulary[END]]
        else: # label 데이터가 아닌 입력 데이터의 경우
            if len(sentence_index) > max_sequences:
                #0이상 ~ 30미만의 데이터 까지 자름
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]#[0]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)    # 빠른 연산을 지원하는 넘파이 배열로 리턴 (주어진 모든 문장별 단어들의 인덱스로 구성된 리스트들을 넘파이 배열로)

In [36]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더(첫 문장) 입력 출력 (12시 땡)
x_encoder[0]


array([434, 306,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [37]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


array([    1,  5831,  4399, 10903,  7838,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [38]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]


array([ 5831,  4399, 10903,  7838,     2,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

<br>
<br>

# 모델 생성

In [39]:
# 임베딩 벡터의 차원을 100으로 설정
# 임베딩 : 단어를 고차원 공간에 위치시키는 표현 방법
    # 비슷한 의미를 가진 단어들이 공간에서 가깝게 위치하도록 하는 것
embedding_dim = 100

In [40]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 각 단어들(10진 정수로 변환)을 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 입력 문장의 각 단어들(encoder_inputs 10진 정수로 변환)을 
# 각각 100번 선형 회귀를 통해서 100 칸의 데이터로 변환
# 선형회귀시 weight들은 학습을 통해서 입력 단어들을 잘 표현 할 수 있는
# 값으로 변환 
encoder_dense = layers.Embedding(
                     len(words), #전체 단어의 개수
                     embedding_dim # 임베딩 결과 칸 수 100
                     )(encoder_inputs) # 입력데이터

# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
# return_state가 True면 상태값 리턴
# encoder_outputs, state_h, state_c = layers.LSTM(
#                                     lstm_hidden_dim,#LSTM 결과 칸수 128
#                                     dropout=0.1, # LSTM 최종 결과의 weight 의 dropout
#                                     recurrent_dropout=0.5,#입력데이터와 곱해지는 weight의 dropout
#                                     return_state=True #hidden state와 cell state 동시 출력
#                                     )(encoder_outputs)

_, state_h, state_c = layers.LSTM(
                                    lstm_hidden_dim,#LSTM 결과 칸수 128
                                    dropout=0.1, # LSTM 최종 결과의 weight 의 dropout
                                    recurrent_dropout=0.5,#입력데이터와 곱해지는 weight의 dropout
                                    return_state=True #hidden state와 cell state 동시 출력
                                    )(encoder_dense)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

In [41]:
#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# decoder의 입력 
decoder_inputs = layers.Input(shape=(None,))

# 입력 문장의 각 단어들(decoder_inputs  10진 정수로 변환)을 
# 각각 100번 선형 회귀를 통해서 100 칸의 데이터로 변환
# 선형회귀시 weight들은 학습을 통해서 입력 단어들을 잘 표현 할 수 있는
# 값으로 변환 
decoder_embedding = layers.Embedding(len(words), embedding_dim)

#decoder_outputs = decoder_embedding(decoder_inputs)
decoder_input_dense = decoder_embedding(decoder_inputs)

# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
# return_state가 True면 상태값 리턴

# return_sequences=True : LSTM 각 셀마다 예측값을 리턴
# LSTM의 각 셀의 예측 값은 찐 값인 y_decode와 비교해서 cost 를 계산

decoder_lstm = layers.LSTM(lstm_hidden_dim, # LSTM 출력 결과 128칸
                           dropout=0.1, # LSTM 최종 결과의 weight 의 dropout
                            recurrent_dropout=0.5,#입력데이터와 곱해지는 weight의 dropout
                            return_state=True, #hidden state와 cell state 동시 출력
                            return_sequences=True # LSTM의 각 셀마다 예측값을 리턴
                           )

# initial_state를 인코더의 상태로 초기화
# decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
#                                      initial_state=encoder_states)

decoder_outputs, _, _ = decoder_lstm(decoder_input_dense,
                                      initial_state=encoder_states)

# 단어의 개수만큼 선형회귀해서 원핫 형식으로 각 단어 인덱스를 출력
# 선형 회귀 결과와 y_decode (단어 개수만큼 원핫 인코딩, 예측해야 할 값)
# 비교해서 cost 계산하고 cost가 0이 되도록 학 습 할것임
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [42]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 인코더와 디코더 2개가 같이 있으므로 
# 인코더와 디코더 입력을 설정
# 인코더의 입력은 Question이 저장된 encoder_inputs
# 디코더의 입력은 <START>+ Answer가 저장된 decoder_inputs

# 찐값 설정 
# 찐 값은 Answer가 원 핫 인코딩된 decoder_outputs
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='adam', # AdamOptimizer learning_rate = 0.001
              loss='categorical_crossentropy',#다중 분류
              metrics=['acc'] # 학습시 정확도 출력
              )    

In [43]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 학습이 끝나고 나서 
# 학습된 encoder의 weight를 이용해서 예측을 실행
encoder_model = models.Model(encoder_inputs, encoder_states)

In [44]:
#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

#학습이 끝나고 나서 학습된 decoder의 weight를 이용해서 학습을 실행
# 학습된 encoder LSTM의 hidden state를 저장할 변수
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))

# 학습된 encoder LSTM의 상태를 저장할 변수
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))

# 학습된 encoder LSTM의 hidden state,
#        encoder LSTM의 상태를 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 학습된 인코딩 레이어 저장
#decoder_outputs = decoder_embedding(decoder_inputs)
decoder_embeding_outputs = decoder_embedding(decoder_inputs)

# 학습된 LSTM 레이어 저장
# decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
#                                                  initial_state=decoder_states_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embeding_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

<br>
<br>

# 훈련 및 테스트

In [45]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [46]:
y_decoder.shape

(11823, 30)

In [47]:
# 인코더와 디코더를 배치 사이즈 만큼씩 리턴하는 클래스 구현

class  ChatSequence(tf.keras.utils.Sequence):
    #객체 생성할때 호출 되는 함수
    # 매개변수 : 
    # x_encoder : Question
    # x_decoder : <START> + Answer
    # y_decoder : x_decoder 다음 단어가 저장 x_decoder 입력시 출력해야할 
    #             다음 단어

    # total : 전체 데이터수
    # batch_size : 1번에 리턴할 데이터 수 (32개)
    def __init__(self, x_encoder, x_decoder,y_decoder, total, batch_size ):
        self.x_encoder = x_encoder
        self.x_decoder = x_decoder
        self.y_decoder = y_decoder
        self.total = total
        self.batch_size = batch_size
    
    # batch_size씩 이미지를 리턴했을때 전체 이미지를 리턴하려명 몆번 반복해야 하는지 리턴
    def __len__(self):
        
        # math.ceil : 소숫점 1자리 올림 예) 6.1 -> 7   6 -> 6   6.0 -> 6

        # total (전체 데이터수) / batch_size (한번에 리턴할 이미지 개수) 의 올림을 리턴     
        return math.ceil(self.total / self.batch_size)

    # 학습시 batch_size 씩 이미지를 리턴하는 함수로 텐서플로우에서 
    # 학습시 model 객체에서 자동으로 호출하는 함수
    # 매개변수 idx : 몆번째 batch 인지가 저장되는 매개변수 0부터 시작
    def __getitem__(self, idx):
        #Question을 저장할 리스트
        x_encoder_list = []
        # <START> Answer 를 저장할 리스트
        x_decoder_list = []
        # x_decoder 다음 단어가 저장 x_decoder 입력시 출력해야할 
        # 다음 단어 를 저장할 리스트
        y_decoder_list = []
        #리턴할 시작 인덱스
        start_index = int(idx * self.batch_size)
        # 리턴할 마지막 인덱스
        end_index = int((idx +1)*self.batch_size )

        #start_index ~ end_index 까지 반복       
        for index in range(start_index , end_index):
            #인덱스가 total(전체 데이터보다 작으면)
            if index < self.total :
                #x_encoder 추가            
                x_encoder_list.append(x_encoder[index])
                #x_decoder 추가
                x_decoder_list.append(x_decoder[index])
                #y_decoder 추가
                y_decoder_list.append(y_decoder[index])
            
        # 원핫인코딩 초기화
        one_hot_arr = np.zeros(
                               (len(y_decoder_list), #데이터 수 
                                max_sequences, # 전체 줄수 (30줄)
                                len(words)) #전체 칸수 (단어 개수)
                               )

        # 디코더 목표를 원핫인코딩으로 변환
        # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임

        # y_decoder_list 에 인덱스 추가
        # 인덱스는 i, y_decoder_list에 1개는 sequence에 대입
        for i, sequence in enumerate(y_decoder_list):
            # sequence (y_decoder 1개 , y_decoder는 x_decoder가 예측 할값)
            # 에서 숫자1개를 index에 대입
            # 몆번째 sequence인지 는 j에 대입
            for j, index in enumerate(sequence):
                # i번째 y_decoder , j번째줄, index 번째 칸에 1대입 
                one_hot_arr[i, j, index] = 1

        #np.array(x_encoder_list, dtype="float32") : x_encoder_list 를 배열로 변환
        x_encoder_arr = np.array(x_encoder_list, dtype="float32")
        #np.array(x_decoder_list, dtype="float32") : x_decoder_list를 배열로 변환
        x_decoder_arr = np.array(x_decoder_list, dtype="float32")
        # one_hot_arr을 float32 타입으로 변환
        one_hot_arr = np.array(one_hot_arr, dtype="float32")
        return [x_encoder_arr, x_decoder_arr], one_hot_arr
                

In [48]:
sequence = ChatSequence(x_encoder, x_decoder,y_decoder, len(y_decoder), 32)

In [49]:
for [enc,dec], y_dec in sequence:
    break

In [50]:
enc

array([[  434.,   306.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.],
       [ 8462.,  9448.,  6672.,  8791.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.],
       [ 9291., 11360., 11645.,  8303.,    54., 11460.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.],
       [ 9291., 11360., 11645.,  5871.,  8303.,    54., 11460.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0

In [51]:
dec

array([[1.0000e+00, 5.8310e+03, 4.3990e+03, 1.0903e+04, 7.8380e+03,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 8.3090e+03, 4.0480e+03, 1.0157e+04, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 8.6030e+03, 3.0530e+03, 7.9200e+03, 1.1119e+04,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.

In [52]:
y_dec

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [53]:
# 에폭 반복
for epoch in range(5):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit(sequence,
                        epochs=30,
                        verbose=1)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    

Total Epoch : 1
Epoch 1/30
370/370 [==============================] - 37s 93ms/step - loss: 1.9569 - acc: 0.7979
Epoch 2/30
370/370 [==============================] - 34s 92ms/step - loss: 1.2524 - acc: 0.8231
Epoch 3/30
370/370 [==============================] - 35s 94ms/step - loss: 1.1806 - acc: 0.8282
Epoch 4/30
370/370 [==============================] - 35s 94ms/step - loss: 1.1332 - acc: 0.8329
Epoch 5/30
370/370 [==============================] - 35s 94ms/step - loss: 1.0972 - acc: 0.8359
Epoch 6/30
370/370 [==============================] - 35s 95ms/step - loss: 1.0627 - acc: 0.8400
Epoch 7/30
370/370 [==============================] - 35s 95ms/step - loss: 1.0272 - acc: 0.8441
Epoch 8/30
370/370 [==============================] - 35s 96ms/step - loss: 0.9903 - acc: 0.8479
Epoch 9/30
370/370 [==============================] - 36s 96ms/step - loss: 0.9544 - acc: 0.8507
Epoch 10/30
370/370 [==============================] - 35s 96ms/step - loss: 0.9221 - acc: 0.8532
Epoch 11/30
3

In [54]:
# 모델 저장
encoder_model.save( 'seq2seq_chatbot_encoder_model.h5')
decoder_model.save( 'seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open( 'word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open( 'index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)    